# 🗳️ วิเคราะห์ข้อมูลการเลือกตั้ง 69 จาก กกต. (ECT Election 69 Analysis)

## Data Pipeline & Deep Analysis
ข้อมูลจาก **7 API endpoints** ของ กกต. (Election Commission of Thailand)

### กลุ่มข้อมูลอ้างอิง (Static Data)
| # | Endpoint | คำอธิบาย |
|---|----------|----------|
| 1 | `info_province` | รายชื่อจังหวัดทั้งหมด |
| 2 | `info_constituency` | รายละเอียดเขตเลือกตั้ง |
| 3 | `info_party_overview` | รายชื่อพรรคการเมือง + โลโก้ |
| 4 | `info_mp_candidate` | ผู้สมัคร สส.แบ่งเขต |
| 5 | `info_party_candidate` | ผู้สมัคร สส.บัญชีรายชื่อ |

### กลุ่มผลคะแนน (Live Stats)
| # | Endpoint | คำอธิบาย |
|---|----------|----------|
| 6 | `stats_cons` | ผลคะแนน สส.แบ่งเขต |
| 7 | `stats_party` | ผลคะแนน สส.บัญชีรายชื่อ |

In [2]:
# === 1. Import Libraries ===
import json
import os
import warnings
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

DATA_DIR = Path("../data/ect_api")
print(f"📁 Data directory: {DATA_DIR.resolve()}")
print(f"📅 Analysis time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📁 Data directory: /Users/teerapatcheung/Desktop/0-Project/vending-machine-ttfts/election_69_analyzer/data/ect_api
📅 Analysis time: 2026-02-12 15:24:59


In [4]:
# === 1b. Visualization imports (may take time for font cache) ===
import matplotlib
matplotlib.use('agg')  # non-interactive backend for faster init
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Thai font support
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Sarabun', 'Tahoma', 'TH Sarabun New', 'Angsana New', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100

sns.set_theme(style="whitegrid", font_scale=1.2)
print("✅ Visualization libraries loaded")

✅ Visualization libraries loaded


## 2. โหลดข้อมูลจาก JSON files ทั้ง 7 ชุด

In [3]:
# === 2. Load all 7 JSON datasets ===
def load_json(filename):
    """Load JSON file from data/ect_api/"""
    filepath = DATA_DIR / filename
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

# Load all datasets
raw_province = load_json("info_province.json")
raw_constituency = load_json("info_constituency.json")
raw_party = load_json("info_party_overview.json")
raw_mp_candidate = load_json("info_mp_candidate.json")
raw_party_candidate = load_json("info_party_candidate.json")
raw_stats_cons = load_json("stats_cons.json")
raw_stats_party = load_json("stats_party.json")

# Quick overview
datasets = {
    "info_province": raw_province,
    "info_constituency": raw_constituency,
    "info_party_overview": raw_party,
    "info_mp_candidate": raw_mp_candidate,
    "info_party_candidate": raw_party_candidate,
    "stats_cons": raw_stats_cons,
    "stats_party": raw_stats_party,
}

for name, data in datasets.items():
    if isinstance(data, list):
        print(f"📦 {name}: {len(data)} records (list)")
    elif isinstance(data, dict):
        print(f"📦 {name}: {len(data)} top-level keys → {list(data.keys())[:5]}...")
    else:
        print(f"📦 {name}: type={type(data)}")

📦 info_province: 3 top-level keys → ['total_registered_vote', 'total_vote_stations', 'province']...
📦 info_constituency: 401 records (list)
📦 info_party_overview: 57 records (list)
📦 info_mp_candidate: 3491 records (list)
📦 info_party_candidate: 57 records (list)
📦 stats_cons: 21 top-level keys → ['turn_out', 'percent_turn_out', 'valid_votes', 'invalid_votes', 'blank_votes']...
📦 stats_party: 4 top-level keys → ['counted_vote_stations', 'percent_count', 'result_party', 'last_update']...


## 3. แปลงข้อมูลอ้างอิง (Reference Data) เป็น DataFrames

In [5]:
# === 3. Parse Reference Data into DataFrames ===

# 3a. Provinces (จังหวัด)
provinces_list = raw_province.get("province", raw_province) if isinstance(raw_province, dict) else raw_province
df_provinces = pd.DataFrame(provinces_list)
print(f"🏛️ จังหวัด: {len(df_provinces)} จังหวัด")
print(f"   Columns: {list(df_provinces.columns)}")
df_provinces.head(3)

🏛️ จังหวัด: 77 จังหวัด
   Columns: ['province_id', 'prov_id', 'province', 'abbre_thai', 'eng', 'total_vote_stations', 'total_registered_vote']


,province_id,prov_id,province,abbre_thai,eng,total_vote_stations,total_registered_vote
0,10,BKK,กรุงเทพมหานคร,กทม,BKK,6757,4502458
1,11,SPK,สมุทรปราการ,สป,SPK,1771,1126420
2,12,NBI,นนทบุรี,นบ,NBI,1559,1094383


In [6]:
# 3b. Parties (พรรคการเมือง)
df_parties = pd.DataFrame(raw_party)
df_parties['id'] = df_parties['id'].astype(int)
df_parties['party_no'] = df_parties['party_no'].astype(int)
print(f"🏳️ พรรคการเมือง: {len(df_parties)} พรรค")
df_parties.head(5)

🏳️ พรรคการเมือง: 57 พรรค


,id,party_no,name,abbr,color,logo_url
0,254,1,ไทยทรัพย์ทวี,ททว.,#f02d30,https://static-ectreport69.ect.go.th/data/part...
1,185,2,เพื่อชาติไทย,พ.ชต.ท.,#724a34,https://static-ectreport69.ect.go.th/data/part...
2,244,3,ใหม่,ม,#da3731,https://static-ectreport69.ect.go.th/data/part...
3,228,4,มิติใหม่,มต.,#abc29d,https://static-ectreport69.ect.go.th/data/part...
4,250,5,รวมใจไทย,ร.จ.ท.,#eec49a,https://static-ectreport69.ect.go.th/data/part...


In [7]:
# 3c. Constituencies (เขตเลือกตั้ง)
df_constituency = pd.DataFrame(raw_constituency)
print(f"📍 เขตเลือกตั้ง: {len(df_constituency)} เขต")
df_constituency.head(3)

📍 เขตเลือกตั้ง: 401 เขต


,cons_id,cons_no,prov_id,total_vote_stations,registered_vote,zone
0,BKK_0,0,BKK,95,NaN,NaN
1,BKK_1,1,BKK,239,130423.0,"[เขตพระนคร, เขตดุสิต (ยกเว้นถนนนครไชยศรี), เขต..."
2,BKK_2,2,BKK,232,136481.0,"[เขตปทุมวัน, เขตสาทร, เขตราชเทวี]"


In [8]:
# 3d. MP Candidates (ผู้สมัคร สส.แบ่งเขต)
df_mp_candidates = pd.DataFrame(raw_mp_candidate)
print(f"👤 ผู้สมัคร สส.แบ่งเขต: {len(df_mp_candidates)} คน")
# Extract province code from mp_app_id (e.g., "BKK_1_1" -> "BKK")
df_mp_candidates['prov_id'] = df_mp_candidates['mp_app_id'].str.extract(r'^([A-Z]+)_')
df_mp_candidates['cons_no'] = df_mp_candidates['mp_app_id'].str.extract(r'_(\d+)_\d+$').astype(int)
df_mp_candidates.head(3)

👤 ผู้สมัคร สส.แบ่งเขต: 3491 คน


,mp_app_id,mp_app_no,mp_app_party_id,mp_app_name,image_url,prov_id,cons_no
0,KBI_1_1,1,34.0,นายสุวรรณ อินพรหม,https://static-ectreport69.ect.go.th/data/cand...,KBI,1
1,KBI_1_2,2,63.0,นายกิตติ กิตติธรกุล,https://static-ectreport69.ect.go.th/data/cand...,KBI,1
2,KBI_1_3,3,129.0,นายธนวัช ภูเก้าล้วน,https://static-ectreport69.ect.go.th/data/cand...,KBI,1


## 4. แปลงข้อมูลผลคะแนน (Stats Data) และ Join ข้อมูลทั้งหมด

In [12]:
# === 4a. Parse Stats & Build Joined DataFrames ===

# National summary (everything except result_province)
national = {k: v for k, v in raw_stats_cons.items() if k != 'result_province'}
print("📊 สรุปภาพรวมระดับชาติ:")
for k, v in national.items():
    print(f"   {k}: {v}")

# Province-level stats from stats_cons
province_stats = raw_stats_cons.get('result_province', [])
print(f"\n📊 จำนวนจังหวัดในผลคะแนน: {len(province_stats)}")

# Build province-level & constituency-level DataFrames
prov_records = []
cons_records = []
cand_vote_records = []

for prov in province_stats:
    prov_id = prov['prov_id']
    prov_records.append({
        'prov_id': prov_id,
        'turn_out': prov.get('turn_out', 0),
        'percent_turn_out': prov.get('percent_turn_out', 0),
        'valid_votes': prov.get('valid_votes', 0),
        'invalid_votes': prov.get('invalid_votes', 0),
        'blank_votes': prov.get('blank_votes', 0),
        'party_list_turn_out': prov.get('party_list_turn_out', 0),
        'party_list_valid_votes': prov.get('party_list_valid_votes', 0),
        'counted_vote_stations': prov.get('counted_vote_stations', 0),
        'percent_count': prov.get('percent_count', 0),
        'total_registered_vote': prov.get('total_registered_vote', 0),
    })

    # Key is "constituencies" in actual API data
    for cons in prov.get('constituencies', []):
        cons_id = cons.get('cons_id', '')
        cons_records.append({
            'prov_id': prov_id,
            'cons_id': cons_id,
            'turn_out': cons.get('turn_out', 0),
            'percent_turn_out': cons.get('percent_turn_out', 0),
            'valid_votes': cons.get('valid_votes', 0),
            'invalid_votes': cons.get('invalid_votes', 0),
            'blank_votes': cons.get('blank_votes', 0),
            'counted_vote_stations': cons.get('counted_vote_stations', 0),
            'percent_count': cons.get('percent_count', 0),
        })

        # Key is "candidates" in actual API data
        for cand in cons.get('candidates', []):
            cand_vote_records.append({
                'prov_id': prov_id,
                'cons_id': cons_id,
                'mp_app_id': cand.get('mp_app_id', ''),
                'mp_app_vote': cand.get('mp_app_vote', 0),
                'mp_app_vote_percent': cand.get('mp_app_vote_percent', 0),
                'mp_app_rank': cand.get('mp_app_rank', 0),
                'party_id': cand.get('party_id', None),
            })

df_prov_stats = pd.DataFrame(prov_records)
df_cons_stats = pd.DataFrame(cons_records)
df_cand_votes = pd.DataFrame(cand_vote_records)

print(f"\n✅ Province stats: {df_prov_stats.shape}")
print(f"✅ Constituency stats: {df_cons_stats.shape}")
print(f"✅ Candidate votes: {df_cand_votes.shape}")
df_prov_stats.head(3)

📊 สรุปภาพรวมระดับชาติ:
   turn_out: 34632581
   percent_turn_out: 65.43966
   valid_votes: 31951912
   invalid_votes: 1234047
   blank_votes: 1446622
   percent_valid_votes: 92.25969
   percent_invalid_votes: 3.56325
   percent_blank_votes: 4.17706
   party_list_turn_out: 34565642
   party_list_percent_turn_out: 65.31318
   party_list_valid_votes: 31998836
   party_list_invalid_votes: 1556201
   party_list_blank_votes: 1010605
   party_percent_valid_votes: 92.57411
   party_percent_invalid_votes: 4.50216
   party_percent_blank_votes: 2.92373
   counted_vote_stations: 95583
   percent_count: 94.3275
   pause_report: False
   last_update: 2026-02-10T10:45:50.294Z

📊 จำนวนจังหวัดในผลคะแนน: 77

✅ Province stats: (77, 11)
✅ Constituency stats: (401, 9)
✅ Candidate votes: (3491, 7)


,prov_id,turn_out,percent_turn_out,valid_votes,invalid_votes,blank_votes,party_list_turn_out,party_list_valid_votes,counted_vote_stations,percent_count,total_registered_vote
0,BKK,2921346,64.88336,2717431,48559,155356,2917995,2784699,6289,93.07385,4502458
1,SPK,750303,66.60952,688634,21038,40631,752183,705047,1678,94.74873,1126420
2,NBI,730832,66.78028,668274,17102,45456,730093,688945,1442,92.49519,1094383


In [13]:
# 4b. Build candidate-level results (ผลคะแนนรายบุคคล) — use df_cand_votes from above
df_cand_results = df_cand_votes.copy()
print(f"✅ Candidate results: {df_cand_results.shape[0]} records")

# Join with candidate names
df_cand_full = df_cand_results.merge(
    df_mp_candidates[['mp_app_id', 'mp_app_name', 'mp_app_no', 'mp_app_party_id']],
    on='mp_app_id', how='left'
)

# Join with party names
df_cand_full = df_cand_full.merge(
    df_parties[['id', 'name', 'color', 'party_no']].rename(columns={'id': 'party_id_ref', 'name': 'party_name'}),
    left_on='party_id', right_on='party_id_ref', how='left'
)

# Join with province names
df_cand_full = df_cand_full.merge(
    df_provinces[['prov_id', 'province']],
    on='prov_id', how='left'
)

print(f"✅ Full candidate results (joined): {df_cand_full.shape}")
df_cand_full.head(5)

✅ Candidate results: 3491 records
✅ Full candidate results (joined): (3491, 15)


,prov_id,cons_id,mp_app_id,mp_app_vote,mp_app_vote_percent,mp_app_rank,party_id,mp_app_name,mp_app_no,mp_app_party_id,party_id_ref,party_name,color,party_no,province
0,BKK,BKK_1,BKK_1_5,32564,44.39294,1,129.0,นายปารเมศ วิทยารักษ์สรรค์,5,129.0,129.0,ประชาชน,#fd6512,46.0,กรุงเทพมหานคร
1,BKK,BKK_1,BKK_1_9,14018,19.11007,2,1.0,นายพีรวุฒิ พิมพ์สมฤดี,9,1.0,1.0,ประชาธิปัตย์,#06aff3,27.0,กรุงเทพมหานคร
2,BKK,BKK_1,BKK_1_1,13661,18.62339,3,63.0,นางสาวลลิดา เพริศวิวัฒนา,1,63.0,63.0,ภูมิใจไทย,#0c149c,37.0,กรุงเทพมหานคร
3,BKK,BKK_1,BKK_1_8,5761,7.85370,4,34.0,นายญาณกิตติ์ ห่วงทรัพย์,8,34.0,34.0,เพื่อไทย,#da3731,9.0,กรุงเทพมหานคร
4,BKK,BKK_1,BKK_1_6,1983,2.70333,5,229.0,นายพลัฏฐ์ ศิริกุลพิสุทธิ์,6,229.0,229.0,รวมไทยสร้างชาติ,#020C54,6.0,กรุงเทพมหานคร


In [14]:
# 4d. Party-list stats (ผลคะแนน สส.บัญชีรายชื่อ)
party_results = raw_stats_party.get('result_party', [])
df_party_stats = pd.DataFrame(party_results)

# Join with party names
df_party_full = df_party_stats.merge(
    df_parties[['id', 'name', 'color', 'party_no', 'abbr']].rename(columns={'id': 'party_id'}),
    on='party_id', how='left'
)
df_party_full = df_party_full.sort_values('party_vote', ascending=False).reset_index(drop=True)
print(f"✅ Party-list results: {len(df_party_full)} พรรค")

# Top 15 parties
print("\n🏆 Top 15 พรรค (คะแนนบัญชีรายชื่อ):")
df_party_full[['name', 'party_vote', 'party_vote_percent', 'party_list_count', 
               'first_mp_app_count']].head(15)

✅ Party-list results: 57 พรรค

🏆 Top 15 พรรค (คะแนนบัญชีรายชื่อ):


,name,party_vote,party_vote_percent,party_list_count,first_mp_app_count
0,ประชาชน,9802658,28.30473,None,87
1,ภูมิใจไทย,5964814,17.22313,None,174
2,เพื่อไทย,5158066,14.89368,None,58
3,ประชาธิปัตย์,3662606,10.57561,None,10
4,เศรษฐกิจ,1047721,3.02525,None,0
5,รวมไทยสร้างชาติ,706918,2.04119,None,0
6,เพื่อชาติไทย,629628,1.81802,None,0
7,กล้าธรรม,606312,1.75070,None,56
8,รวมใจไทย,404166,1.16701,None,0
9,ประชาชาติ,401636,1.15971,None,4


---
## 5. 📊 วิเคราะห์: อันดับพรรคการเมือง (คะแนนบัญชีรายชื่อ)
คะแนนรวม Party-List ทั่วประเทศ — พรรคไหนได้คะแนนมากที่สุด?

In [15]:
# === 5. Party Rankings (บัญชีรายชื่อ) ===
top_n = 15
df_top_parties = df_party_full.head(top_n).copy()

fig, ax = plt.subplots(figsize=(14, 8))
colors = df_top_parties['color'].fillna('#888888').tolist()
bars = ax.barh(
    range(top_n), 
    df_top_parties['party_vote'], 
    color=colors, 
    edgecolor='white', 
    linewidth=0.5
)
ax.set_yticks(range(top_n))
ax.set_yticklabels(df_top_parties['name'].tolist())
ax.invert_yaxis()
ax.set_xlabel('คะแนนบัญชีรายชื่อ (Party-List Votes)')
ax.set_title(f'🏆 Top {top_n} พรรคการเมือง — คะแนนบัญชีรายชื่อทั่วประเทศ\n(นับแล้ว {national.get("percent_count", 0):.1f}%)', fontsize=16)

# Add vote count labels
for i, (bar, vote) in enumerate(zip(bars, df_top_parties['party_vote'])):
    pct = df_top_parties.iloc[i]['party_vote_percent']
    ax.text(bar.get_width() + max(df_top_parties['party_vote']) * 0.01, bar.get_y() + bar.get_height()/2, 
            f'{vote:,.0f} ({pct:.1f}%)', va='center', fontsize=10)

ax.set_xlim(0, max(df_top_parties['party_vote']) * 1.25)
plt.tight_layout()
plt.show()

---
## 6. 🏅 วิเคราะห์: พรรคที่ชนะเขตเลือกตั้งมากที่สุด (สส.แบ่งเขต)
นับจำนวนเขตที่ผู้สมัครของแต่ละพรรคได้คะแนนอันดับ 1

In [16]:
# === 6. Constituency Winners (พรรคที่ชนะแบ่งเขต) ===

# Filter rank 1 candidates (winners in each constituency)
df_winners = df_cand_full[df_cand_full['mp_app_rank'] == 1].copy()
print(f"🏅 จำนวนเขตที่มีผู้ชนะ: {len(df_winners)} เขต")

# Count wins by party
party_wins = df_winners.groupby(['party_name', 'color']).size().reset_index(name='seats_won')
party_wins = party_wins.sort_values('seats_won', ascending=False).reset_index(drop=True)

# Visualize
fig, ax = plt.subplots(figsize=(14, 8))
top_win = party_wins.head(15)
colors_win = top_win['color'].fillna('#888888').tolist()
bars = ax.barh(range(len(top_win)), top_win['seats_won'], color=colors_win, edgecolor='white')
ax.set_yticks(range(len(top_win)))
ax.set_yticklabels(top_win['party_name'].tolist())
ax.invert_yaxis()
ax.set_xlabel('จำนวนเขตที่ชนะ')
ax.set_title('🏅 พรรคที่ชนะเขตเลือกตั้งมากที่สุด (สส.แบ่งเขต)', fontsize=16)

for bar, seats in zip(bars, top_win['seats_won']):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{seats} เขต', va='center', fontsize=11, fontweight='bold')

ax.set_xlim(0, max(top_win['seats_won']) * 1.2)
plt.tight_layout()
plt.show()

print(f"\n📊 สรุปจำนวนเขตที่ชนะ:")
party_wins[['party_name', 'seats_won']].head(10)

🏅 จำนวนเขตที่มีผู้ชนะ: 400 เขต

📊 สรุปจำนวนเขตที่ชนะ:


,party_name,seats_won
0,ภูมิใจไทย,174
1,ประชาชน,87
2,เพื่อไทย,58
3,กล้าธรรม,56
4,ประชาธิปัตย์,10
5,ไทรวมพลัง,5
6,ประชาชาติ,4
7,พลังประชารัฐ,4
8,โอกาสใหม่,1
9,ไทยสร้างไทย,1


---
## 7. 📈 วิเคราะห์: จำนวน สส. รวม (แบ่งเขต + บัญชีรายชื่อ)
ประมาณการจำนวน สส. ทั้งแบ่งเขต (400 คน) + บัญชีรายชื่อ (100 คน) = 500 คน

In [17]:
# === 7. Combined MP Estimation (แบ่งเขต + บัญชีรายชื่อ) ===

# Constituency seats (from actual winners)
cons_seats = party_wins[['party_name', 'seats_won']].rename(columns={'seats_won': 'constituency_seats'})

# Party-list seat estimation (simplified: 100 seats proportional)
total_party_votes = df_party_full['party_vote'].sum()
df_party_full['estimated_pl_seats'] = (df_party_full['party_vote'] / total_party_votes * 100).apply(np.floor).astype(int)

pl_seats = df_party_full[['name', 'estimated_pl_seats']].rename(columns={'name': 'party_name'})

# Merge
df_combined = cons_seats.merge(pl_seats, on='party_name', how='outer').fillna(0)
df_combined['constituency_seats'] = df_combined['constituency_seats'].astype(int)
df_combined['estimated_pl_seats'] = df_combined['estimated_pl_seats'].astype(int)
df_combined['total_estimated_seats'] = df_combined['constituency_seats'] + df_combined['estimated_pl_seats']
df_combined = df_combined.sort_values('total_estimated_seats', ascending=False).reset_index(drop=True)

# Visualize stacked bar
top_combined = df_combined.head(12)
fig, ax = plt.subplots(figsize=(14, 8))

y_pos = range(len(top_combined))
bars1 = ax.barh(y_pos, top_combined['constituency_seats'], label='สส.แบ่งเขต', color='#2196F3', edgecolor='white')
bars2 = ax.barh(y_pos, top_combined['estimated_pl_seats'], left=top_combined['constituency_seats'], 
                label='สส.บัญชีรายชื่อ (ประมาณการ)', color='#FF9800', edgecolor='white')

ax.set_yticks(y_pos)
ax.set_yticklabels(top_combined['party_name'].tolist())
ax.invert_yaxis()
ax.set_xlabel('จำนวน สส. (ที่นั่ง)')
ax.set_title('📊 ประมาณการจำนวน สส. รวม (แบ่งเขต + บัญชีรายชื่อ)', fontsize=16)
ax.legend(loc='lower right', fontsize=12)

for i, (c, p, t) in enumerate(zip(top_combined['constituency_seats'], 
                                     top_combined['estimated_pl_seats'], 
                                     top_combined['total_estimated_seats'])):
    ax.text(t + 1, i, f'{t} ({c}+{p})', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 ตาราง สส. ประมาณการ:")
df_combined.head(15)


📊 ตาราง สส. ประมาณการ:


,party_name,constituency_seats,estimated_pl_seats,total_estimated_seats
0,ภูมิใจไทย,174,18,192
1,ประชาชน,87,30,117
2,เพื่อไทย,58,16,74
3,กล้าธรรม,56,1,57
4,ประชาธิปัตย์,10,11,21
5,ไทรวมพลัง,5,0,5
6,ประชาชาติ,4,1,5
7,พลังประชารัฐ,4,0,4
8,เศรษฐกิจ,0,3,3
9,รวมไทยสร้างชาติ,0,2,2


---
## 8. 🗳️ วิเคราะห์: ผู้มาใช้สิทธิ (Voter Turnout) รายจังหวัด

In [18]:
# === 8. Voter Turnout by Province ===
df_turnout = df_prov_stats.merge(df_provinces[['prov_id', 'province']], on='prov_id', how='left')
df_turnout = df_turnout.sort_values('percent_turn_out', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(20, 16))

# Top 20 highest turnout
top20_high = df_turnout.tail(20)
axes[0].barh(range(len(top20_high)), top20_high['percent_turn_out'], color='#4CAF50', edgecolor='white')
axes[0].set_yticks(range(len(top20_high)))
axes[0].set_yticklabels(top20_high['province'].tolist())
axes[0].set_xlabel('% ผู้มาใช้สิทธิ')
axes[0].set_title('🟢 Top 20 จังหวัด — ผู้มาใช้สิทธิมากที่สุด', fontsize=14)
for i, pct in enumerate(top20_high['percent_turn_out']):
    axes[0].text(pct + 0.3, i, f'{pct:.1f}%', va='center', fontsize=9)

# Top 20 lowest turnout
top20_low = df_turnout.head(20)
axes[1].barh(range(len(top20_low)), top20_low['percent_turn_out'], color='#F44336', edgecolor='white')
axes[1].set_yticks(range(len(top20_low)))
axes[1].set_yticklabels(top20_low['province'].tolist())
axes[1].set_xlabel('% ผู้มาใช้สิทธิ')
axes[1].set_title('🔴 Top 20 จังหวัด — ผู้มาใช้สิทธิน้อยที่สุด', fontsize=14)
for i, pct in enumerate(top20_low['percent_turn_out']):
    axes[1].text(pct + 0.3, i, f'{pct:.1f}%', va='center', fontsize=9)

plt.suptitle(f'📊 วิเคราะห์ผู้มาใช้สิทธิเลือกตั้งรายจังหวัด (เฉลี่ยทั้งประเทศ: {national.get("percent_turn_out", 0):.1f}%)', 
             fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

---
## 9. 📋 วิเคราะห์: บัตรเสีย & บัตรไม่ประสงค์ลงคะแนน
ตรวจสอบอัตราบัตรเสียรายจังหวัด — จังหวัดไหนมีบัตรเสียมากผิดปกติ?

In [19]:
# === 9. Invalid & Blank Ballot Analysis ===
df_turnout['invalid_pct'] = (df_turnout['invalid_votes'] / df_turnout['turn_out'] * 100).round(2)
df_turnout['blank_pct'] = (df_turnout['blank_votes'] / df_turnout['turn_out'] * 100).round(2)

# Flag anomalies (> 2 std above mean)
mean_inv = df_turnout['invalid_pct'].mean()
std_inv = df_turnout['invalid_pct'].std()
threshold_inv = mean_inv + 2 * std_inv
df_turnout['invalid_anomaly'] = df_turnout['invalid_pct'] > threshold_inv

fig, axes = plt.subplots(1, 2, figsize=(20, 12))

# Invalid ballots
df_inv_sorted = df_turnout.sort_values('invalid_pct', ascending=True).tail(25)
colors_inv = ['#FF0000' if x else '#FF9800' for x in df_inv_sorted['invalid_anomaly']]
axes[0].barh(range(len(df_inv_sorted)), df_inv_sorted['invalid_pct'], color=colors_inv, edgecolor='white')
axes[0].set_yticks(range(len(df_inv_sorted)))
axes[0].set_yticklabels(df_inv_sorted['province'].tolist())
axes[0].axvline(x=mean_inv, color='blue', linestyle='--', label=f'ค่าเฉลี่ย ({mean_inv:.1f}%)')
axes[0].axvline(x=threshold_inv, color='red', linestyle='--', label=f'เกณฑ์ผิดปกติ ({threshold_inv:.1f}%)')
axes[0].set_xlabel('% บัตรเสีย')
axes[0].set_title('🔴 Top 25 จังหวัด — อัตราบัตรเสียสูง', fontsize=14)
axes[0].legend()

# Blank ballots (ไม่ประสงค์ลงคะแนน)
df_blank_sorted = df_turnout.sort_values('blank_pct', ascending=True).tail(25)
axes[1].barh(range(len(df_blank_sorted)), df_blank_sorted['blank_pct'], color='#9E9E9E', edgecolor='white')
axes[1].set_yticks(range(len(df_blank_sorted)))
axes[1].set_yticklabels(df_blank_sorted['province'].tolist())
axes[1].set_xlabel('% บัตรไม่ประสงค์ลงคะแนน')
axes[1].set_title('⚪ Top 25 จังหวัด — อัตราไม่ประสงค์ลงคะแนนสูง', fontsize=14)

plt.suptitle('📋 วิเคราะห์บัตรเสียและบัตรไม่ประสงค์ลงคะแนน', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

# Show anomalies
anomalies = df_turnout[df_turnout['invalid_anomaly']][['province', 'invalid_pct', 'turn_out', 'invalid_votes']]
if len(anomalies) > 0:
    print(f"\n⚠️ จังหวัดที่บัตรเสียสูงผิดปกติ (> {threshold_inv:.1f}%):")
    print(anomalies.sort_values('invalid_pct', ascending=False).to_string(index=False))
else:
    print("✅ ไม่พบจังหวัดที่มีบัตรเสียสูงผิดปกติ")


⚠️ จังหวัดที่บัตรเสียสูงผิดปกติ (> 5.8%):
  province  invalid_pct  turn_out  invalid_votes
       ตาก         7.30    264971          19333
แม่ฮ่องสอน         6.44    134769           8680
       เลย         6.26    323275          20241


---
## 10. 🗺️ วิเคราะห์: ภูมิภาคกับฐานเสียงพรรค (Regional Voting Patterns)
เปรียบเทียบคะแนนพรรคหลักในแต่ละภูมิภาค — พรรคไหนแข็งแกร่งในภาคไหน?

In [20]:
# === 10. Regional Voting Patterns ===

# Map provinces to regions based on province_id
region_map = {}
for _, row in df_provinces.iterrows():
    pid = str(row.get('province_id', ''))
    if pid.startswith('10') or pid.startswith('11') or pid.startswith('12') or pid.startswith('13') or pid.startswith('14'):
        region_map[row['prov_id']] = 'กลาง'
    elif pid.startswith('2'):
        region_map[row['prov_id']] = 'ตะวันออก'
    elif pid.startswith('3') or pid.startswith('4'):
        region_map[row['prov_id']] = 'อีสาน'
    elif pid.startswith('5') or pid.startswith('6'):
        region_map[row['prov_id']] = 'เหนือ'
    elif pid.startswith('7'):
        region_map[row['prov_id']] = 'ตะวันตก'
    elif pid.startswith('8'):
        region_map[row['prov_id']] = 'ใต้'
    elif pid.startswith('9'):
        region_map[row['prov_id']] = 'ใต้'
    else:
        region_map[row['prov_id']] = 'อื่นๆ'

# Add region to winners
df_winners['region'] = df_winners['prov_id'].map(region_map)

# Top parties for regional analysis
top_parties_list = party_wins.head(8)['party_name'].tolist()
df_winners_top = df_winners[df_winners['party_name'].isin(top_parties_list)]

# Pivot: Region x Party = seat count
regional_pivot = pd.crosstab(df_winners_top['region'], df_winners_top['party_name'])
regional_pivot = regional_pivot[top_parties_list]  # Reorder columns

# Heatmap
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Heatmap of absolute seats
sns.heatmap(regional_pivot, annot=True, fmt='d', cmap='YlOrRd', ax=axes[0], 
            linewidths=0.5, cbar_kws={'label': 'จำนวนเขตที่ชนะ'})
axes[0].set_title('🗺️ จำนวนเขตที่ชนะ แยกตามภาค & พรรค', fontsize=14)
axes[0].set_ylabel('ภูมิภาค')
axes[0].set_xlabel('พรรคการเมือง')

# Normalized heatmap (% within each region)
regional_pct = regional_pivot.div(regional_pivot.sum(axis=1), axis=0) * 100
sns.heatmap(regional_pct, annot=True, fmt='.1f', cmap='Blues', ax=axes[1],
            linewidths=0.5, cbar_kws={'label': '% ในภาค'})
axes[1].set_title('🗺️ สัดส่วนเขตที่ชนะ (%) แยกตามภาค & พรรค', fontsize=14)
axes[1].set_ylabel('ภูมิภาค')
axes[1].set_xlabel('พรรคการเมือง')

plt.tight_layout()
plt.show()

---
## 11. 🔍 ตรวจสอบความโปร่งใส: Anomaly Detection (Digital Audit Trail)
วิเคราะห์ร่องรอยดิจิทัลเพื่อตรวจสอบความสอดคล้องของข้อมูล:
1. **ตรวจสอบยอดรวม**: คะแนนรวม + บัตรเสีย + บัตรไม่ลงคะแนน = ผู้มาใช้สิทธิ?
2. **Outlier Turnout**: เขตที่มี turnout สูง/ต่ำผิดปกติ
3. **Benford's Law**: ตรวจสอบการกระจายตัวของตัวเลขหลักแรก

In [21]:
# === 11a. Vote Total Consistency Check ===
# valid_votes + invalid_votes + blank_votes should ≈ turn_out
df_cons_check = df_cons_stats.copy()
df_cons_check['calculated_total'] = df_cons_check['valid_votes'] + df_cons_check['invalid_votes'] + df_cons_check['blank_votes']
df_cons_check['total_diff'] = abs(df_cons_check['calculated_total'] - df_cons_check['turn_out'])
df_cons_check = df_cons_check.merge(df_provinces[['prov_id', 'province']], on='prov_id', how='left')

mismatches = df_cons_check[df_cons_check['total_diff'] > 0]
print(f"🔍 ตรวจสอบความสอดคล้องยอดรวม:")
print(f"   เขตทั้งหมด: {len(df_cons_check)}")
print(f"   เขตที่ยอดไม่ตรง: {len(mismatches)}")
if len(mismatches) > 0:
    print(f"   ผลต่างเฉลี่ย: {mismatches['total_diff'].mean():.1f} เสียง")
    print(f"   ผลต่างสูงสุด: {mismatches['total_diff'].max():.0f} เสียง")
    print("\n   Top 10 เขตที่ยอดต่างมากที่สุด:")
    top_diff = mismatches.nlargest(10, 'total_diff')[['province', 'cons_id', 'turn_out', 'calculated_total', 'total_diff']]
    print(top_diff.to_string(index=False))
else:
    print("   ✅ ยอดรวมตรงกันทุกเขต")

🔍 ตรวจสอบความสอดคล้องยอดรวม:
   เขตทั้งหมด: 401
   เขตที่ยอดไม่ตรง: 0
   ✅ ยอดรวมตรงกันทุกเขต


In [22]:
# 11b. Turnout Outlier Detection (Z-score)
# Filter out cons_id ending with _0 (aggregate rows)
df_cons_analysis = df_cons_check[~df_cons_check['cons_id'].str.endswith('_0')].copy()
df_cons_analysis['turnout_zscore'] = (df_cons_analysis['percent_turn_out'] - df_cons_analysis['percent_turn_out'].mean()) / df_cons_analysis['percent_turn_out'].std()
df_cons_analysis['turnout_anomaly'] = abs(df_cons_analysis['turnout_zscore']) > 2.5

outliers = df_cons_analysis[df_cons_analysis['turnout_anomaly']].sort_values('turnout_zscore')

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Distribution of turnout
axes[0].hist(df_cons_analysis['percent_turn_out'], bins=50, color='#2196F3', edgecolor='white', alpha=0.7)
axes[0].axvline(x=df_cons_analysis['percent_turn_out'].mean(), color='red', linestyle='--', label='ค่าเฉลี่ย')
axes[0].set_xlabel('% ผู้มาใช้สิทธิ')
axes[0].set_ylabel('จำนวนเขต')
axes[0].set_title('📊 การกระจายตัวของ Turnout รายเขต', fontsize=13)
axes[0].legend()

# Z-score scatter
colors_z = ['red' if x else '#2196F3' for x in df_cons_analysis['turnout_anomaly']]
axes[1].scatter(range(len(df_cons_analysis)), df_cons_analysis['turnout_zscore'], c=colors_z, alpha=0.5, s=20)
axes[1].axhline(y=2.5, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(y=-2.5, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('เขตเลือกตั้ง (index)')
axes[1].set_ylabel('Z-score (Turnout)')
axes[1].set_title(f'🔍 Turnout Outliers (|Z| > 2.5): {len(outliers)} เขต', fontsize=13)

plt.tight_layout()
plt.show()

if len(outliers) > 0:
    print(f"\n⚠️ เขตที่ Turnout ผิดปกติ ({len(outliers)} เขต):")
    print(outliers[['province', 'cons_id', 'percent_turn_out', 'turnout_zscore', 'turn_out']].to_string(index=False))


⚠️ เขตที่ Turnout ผิดปกติ (3 เขต):
province cons_id  percent_turn_out  turnout_zscore  turn_out
 ชัยภูมิ   CPM_1          11.52031       -7.961355     14912
   ระยอง   RYG_4          44.57737       -3.080521     57717
 ชัยภูมิ   CPM_4          45.25185       -2.980935     54838


In [23]:
# 11c. Benford's Law Analysis
# First-digit distribution of candidate votes
def first_digit(n):
    """Extract first digit of a number."""
    n = abs(int(n))
    if n == 0:
        return 0
    while n >= 10:
        n //= 10
    return n

# Get all non-zero candidate votes
all_votes = df_cand_full[df_cand_full['mp_app_vote'] > 0]['mp_app_vote']
first_digits = all_votes.apply(first_digit)
digit_counts = first_digits.value_counts().sort_index()
digit_freq = digit_counts / digit_counts.sum() * 100

# Benford's expected distribution
benford_expected = {d: np.log10(1 + 1/d) * 100 for d in range(1, 10)}

fig, ax = plt.subplots(figsize=(12, 6))
x = range(1, 10)
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], [digit_freq.get(d, 0) for d in x], 
               width, label='ข้อมูลจริง (คะแนนผู้สมัคร)', color='#2196F3', edgecolor='white')
bars2 = ax.bar([i + width/2 for i in x], [benford_expected[d] for d in x], 
               width, label="Benford's Law (ค่าที่ควรจะเป็น)", color='#FF9800', edgecolor='white')

ax.set_xlabel('ตัวเลขหลักแรก')
ax.set_ylabel('ความถี่ (%)')
ax.set_title("🔍 Benford's Law Analysis — ตรวจสอบความเป็นธรรมชาติของตัวเลข", fontsize=14)
ax.set_xticks(x)
ax.legend(fontsize=12)

# Add percentage labels
for bar, val in zip(bars1, [digit_freq.get(d, 0) for d in x]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}%', 
            ha='center', fontsize=9)
for bar, val in zip(bars2, [benford_expected[d] for d in x]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}%', 
            ha='center', fontsize=9)

plt.tight_layout()
plt.show()

# Chi-square test
from scipy import stats
observed = np.array([digit_freq.get(d, 0) for d in range(1, 10)])
expected = np.array([benford_expected[d] for d in range(1, 10)])
chi2, p_value = stats.chisquare(observed, expected)
print(f"\n📐 Benford's Law Chi-Square Test:")
print(f"   χ² = {chi2:.4f}")
print(f"   p-value = {p_value:.6f}")
if p_value > 0.05:
    print("   ✅ ข้อมูลสอดคล้องกับ Benford's Law (ไม่พบความผิดปกติทางสถิติ)")
else:
    print("   ⚠️ ข้อมูลไม่สอดคล้องกับ Benford's Law (ควรตรวจสอบเพิ่มเติม)")


📐 Benford's Law Chi-Square Test:
   χ² = 0.6034
   p-value = 0.999729
   ✅ ข้อมูลสอดคล้องกับ Benford's Law (ไม่พบความผิดปกติทางสถิติ)


---
## 12. 📊 สรุปภาพรวมการเลือกตั้ง (Election Summary Dashboard)

---
## 11d. 🔍 ตรวจสอบ: บัตรดี (Valid Votes) vs ผลรวมคะแนนผู้สมัครรายเขต
ในแต่ละเขตเลือกตั้ง **บัตรดี** ควรจะเท่ากับ **ผลรวมคะแนนผู้สมัครทุกคน** ในเขตนั้น  
ถ้าไม่ตรงกัน อาจเป็นเพราะ: ยังนับไม่ครบ, ข้อมูล API ไม่ sync, หรือมีความผิดปกติ

In [26]:
# === 11d. Valid Votes vs Sum of Candidate Votes per Constituency ===

# 1) Sum of all candidate votes per constituency
cand_sum_by_cons = df_cand_full.groupby('cons_id')['mp_app_vote'].sum().reset_index()
cand_sum_by_cons.columns = ['cons_id', 'sum_cand_votes']

# 2) Merge with constituency stats (valid_votes = บัตรดี)
df_vote_check = df_cons_stats[~df_cons_stats['cons_id'].str.endswith('_0')].copy()
df_vote_check = df_vote_check.merge(cand_sum_by_cons, on='cons_id', how='left')
df_vote_check = df_vote_check.merge(df_provinces[['prov_id', 'province']], on='prov_id', how='left')

# 3) Calculate difference
df_vote_check['diff'] = df_vote_check['valid_votes'] - df_vote_check['sum_cand_votes']
df_vote_check['diff_abs'] = df_vote_check['diff'].abs()
df_vote_check['diff_pct'] = (df_vote_check['diff_abs'] / df_vote_check['valid_votes'] * 100).round(4)

# 4) Summary stats
total_cons = len(df_vote_check)
exact_match = (df_vote_check['diff'] == 0).sum()
within_1pct = (df_vote_check['diff_pct'] <= 1).sum()
has_diff = (df_vote_check['diff'] != 0).sum()

print("=" * 70)
print("🔍 ตรวจสอบ: บัตรดี (Valid Votes) vs ผลรวมคะแนนผู้สมัคร สส.แบ่งเขต")
print("=" * 70)
print(f"\n📊 จำนวนเขตทั้งหมด: {total_cons} เขต")
print(f"   ✅ ตรงกันพอดี (diff = 0):   {exact_match} เขต ({exact_match/total_cons*100:.1f}%)")
print(f"   🔶 ต่างกัน (diff ≠ 0):      {has_diff} เขต ({has_diff/total_cons*100:.1f}%)")
print(f"   📏 ต่างกัน ≤ 1%:            {within_1pct} เขต")
print(f"\n   📐 ผลต่างเฉลี่ย: {df_vote_check['diff_abs'].mean():,.1f} เสียง")
print(f"   📐 ผลต่างสูงสุด: {df_vote_check['diff_abs'].max():,.0f} เสียง")
print(f"   📐 ผลรวม diff ทั้งหมด: {df_vote_check['diff'].sum():,.0f} เสียง")

# === VISUALIZATION ===
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# --- Plot 1: Scatter — Valid Votes vs Sum Candidate Votes ---
ax1 = axes[0, 0]
sc = ax1.scatter(df_vote_check['valid_votes'], df_vote_check['sum_cand_votes'],
                 c=df_vote_check['diff_pct'], cmap='RdYlGn_r', s=30, alpha=0.7,
                 edgecolors='gray', linewidth=0.3)
# Perfect line y=x
max_val = max(df_vote_check['valid_votes'].max(), df_vote_check['sum_cand_votes'].max())
ax1.plot([0, max_val], [0, max_val], 'r--', linewidth=1.5, label='เส้น y=x (ตรงกันพอดี)')
ax1.set_xlabel('บัตรดี (Valid Votes)')
ax1.set_ylabel('ผลรวมคะแนนผู้สมัคร')
ax1.set_title('📊 บัตรดี vs ผลรวมคะแนนผู้สมัคร (สส.แบ่งเขต)', fontsize=14)
ax1.legend(fontsize=11)
plt.colorbar(sc, ax=ax1, label='% ผลต่าง')

# --- Plot 2: Histogram of differences ---
ax2 = axes[0, 1]
diff_nonzero = df_vote_check[df_vote_check['diff'] != 0]['diff']
if len(diff_nonzero) > 0:
    ax2.hist(diff_nonzero, bins=50, color='#FF5722', edgecolor='white', alpha=0.8)
    ax2.axvline(x=0, color='black', linestyle='-', linewidth=2, label='ไม่มีผลต่าง')
    ax2.axvline(x=diff_nonzero.mean(), color='blue', linestyle='--',
                label=f'ค่าเฉลี่ย ({diff_nonzero.mean():,.0f})')
    ax2.set_xlabel('ผลต่าง (บัตรดี − คะแนนรวม)')
    ax2.set_ylabel('จำนวนเขต')
    ax2.set_title(f'📊 การกระจายตัวของผลต่าง ({len(diff_nonzero)} เขตที่ไม่ตรง)', fontsize=14)
    ax2.legend()
else:
    ax2.text(0.5, 0.5, '✅ ทุกเขตตรงกันพอดี!', transform=ax2.transAxes,
             fontsize=18, ha='center', va='center', color='green')
    ax2.set_title('📊 การกระจายตัวของผลต่าง', fontsize=14)

# --- Plot 3: Top 20 constituencies with biggest difference ---
ax3 = axes[1, 0]
top20_diff = df_vote_check.nlargest(20, 'diff_abs').sort_values('diff_abs', ascending=True)
label_col = top20_diff['province'] + ' (' + top20_diff['cons_id'] + ')'
colors_diff = ['#F44336' if d > 0 else '#2196F3' for d in top20_diff['diff']]
ax3.barh(range(len(top20_diff)), top20_diff['diff'], color=colors_diff, edgecolor='white')
ax3.set_yticks(range(len(top20_diff)))
ax3.set_yticklabels(label_col.tolist(), fontsize=9)
ax3.axvline(x=0, color='black', linewidth=1)
ax3.set_xlabel('ผลต่าง (เสียง) — แดง: บัตรดี > คะแนนรวม, น้ำเงิน: บัตรดี < คะแนนรวม')
ax3.set_title('🏆 Top 20 เขตที่มีผลต่างมากที่สุด', fontsize=14)
for i, (d, pct) in enumerate(zip(top20_diff['diff'], top20_diff['diff_pct'])):
    ax3.text(d + (50 if d >= 0 else -50), i, f'{d:+,.0f} ({pct:.2f}%)',
             va='center', fontsize=8, ha='left' if d >= 0 else 'right')

# --- Plot 4: diff_pct distribution (% ผลต่าง) ---
ax4 = axes[1, 1]
ax4.hist(df_vote_check['diff_pct'], bins=40, color='#9C27B0', edgecolor='white', alpha=0.8)
ax4.axvline(x=1, color='red', linestyle='--', label='เกณฑ์ 1%')
ax4.set_xlabel('% ผลต่าง (|diff| / valid_votes × 100)')
ax4.set_ylabel('จำนวนเขต')
ax4.set_title('📊 การกระจายตัวของ % ผลต่าง', fontsize=14)
ax4.legend()

plt.suptitle('🔍 ตรวจสอบความสอดคล้อง: บัตรดี vs ผลรวมคะแนนผู้สมัคร สส.แบ่งเขต',
             fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

# --- Detail Table ---
print("\n" + "=" * 70)
print("📋 ตาราง: เขตที่ผลต่างมากที่สุด 15 อันดับ")
print("=" * 70)
top15 = df_vote_check.nlargest(15, 'diff_abs')[
    ['province', 'cons_id', 'valid_votes', 'sum_cand_votes', 'diff', 'diff_pct', 'percent_count']
].copy()
top15.columns = ['จังหวัด', 'เขต', 'บัตรดี', 'คะแนนรวมผู้สมัคร', 'ผลต่าง', '%ผลต่าง', '%นับแล้ว']
print(top15.to_string(index=False))

🔍 ตรวจสอบ: บัตรดี (Valid Votes) vs ผลรวมคะแนนผู้สมัคร สส.แบ่งเขต

📊 จำนวนเขตทั้งหมด: 400 เขต
   ✅ ตรงกันพอดี (diff = 0):   400 เขต (100.0%)
   🔶 ต่างกัน (diff ≠ 0):      0 เขต (0.0%)
   📏 ต่างกัน ≤ 1%:            400 เขต

   📐 ผลต่างเฉลี่ย: 0.0 เสียง
   📐 ผลต่างสูงสุด: 0 เสียง
   📐 ผลรวม diff ทั้งหมด: 0 เสียง

📋 ตาราง: เขตที่ผลต่างมากที่สุด 15 อันดับ
      จังหวัด    เขต  บัตรดี  คะแนนรวมผู้สมัคร  ผลต่าง  %ผลต่าง  %นับแล้ว
กรุงเทพมหานคร  BKK_1   73354             73354       0      0.0  94.97908
กรุงเทพมหานคร  BKK_2   78713             78713       0      0.0  94.82759
กรุงเทพมหานคร  BKK_3   72671             72671       0      0.0  94.93088
กรุงเทพมหานคร  BKK_4   70338             70338       0      0.0  94.92386
กรุงเทพมหานคร  BKK_5   86672             86672       0      0.0  94.93088
กรุงเทพมหานคร  BKK_6   89947             89947       0      0.0  95.00000
กรุงเทพมหานคร  BKK_7   80487             80487       0      0.0  94.81132
กรุงเทพมหานคร  BKK_8   86197             86197       0 

In [27]:
# === 11e. Party-List: Valid Votes vs Sum of Party Votes per Constituency ===
# เช็คเพิ่ม: คะแนนบัญชีรายชื่อ (party_list_valid_votes) ระดับจังหวัด
# vs ผลรวมคะแนนทุกพรรค (result_party.party_list_vote) ในแต่ละเขต

# Parse party-list votes per constituency from raw data
pl_cons_records = []
for prov in province_stats:
    prov_id = prov['prov_id']
    for cons in prov.get('constituencies', []):
        cons_id = cons.get('cons_id', '')
        total_pl_vote = sum(p.get('party_list_vote', 0) for p in cons.get('result_party', []))
        pl_cons_records.append({
            'prov_id': prov_id,
            'cons_id': cons_id,
            'pl_valid_votes': cons.get('party_list_valid_votes', 0),
            'sum_party_pl_votes': total_pl_vote,
        })

df_pl_check = pd.DataFrame(pl_cons_records)
df_pl_check = df_pl_check[~df_pl_check['cons_id'].str.endswith('_0')].copy()
df_pl_check = df_pl_check.merge(df_provinces[['prov_id', 'province']], on='prov_id', how='left')
df_pl_check['diff'] = df_pl_check['pl_valid_votes'] - df_pl_check['sum_party_pl_votes']
df_pl_check['diff_abs'] = df_pl_check['diff'].abs()
df_pl_check['diff_pct'] = np.where(
    df_pl_check['pl_valid_votes'] > 0,
    (df_pl_check['diff_abs'] / df_pl_check['pl_valid_votes'] * 100).round(4),
    0
)

# Summary
total_pl = len(df_pl_check)
exact_pl = (df_pl_check['diff'] == 0).sum()
has_diff_pl = (df_pl_check['diff'] != 0).sum()

print("=" * 70)
print("🔍 ตรวจสอบ: บัตรดีบัญชีรายชื่อ vs ผลรวมคะแนนทุกพรรค (รายเขต)")
print("=" * 70)
print(f"\n📊 จำนวนเขต: {total_pl} เขต")
print(f"   ✅ ตรงกันพอดี: {exact_pl} เขต ({exact_pl/total_pl*100:.1f}%)")
print(f"   🔶 ต่างกัน:    {has_diff_pl} เขต ({has_diff_pl/total_pl*100:.1f}%)")
if has_diff_pl > 0:
    print(f"   📐 ผลต่างเฉลี่ย: {df_pl_check['diff_abs'].mean():,.1f} เสียง")
    print(f"   📐 ผลต่างสูงสุด: {df_pl_check['diff_abs'].max():,.0f} เสียง")

# === COMBINED VISUALIZATION ===
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Left: Constituency (MP) — scatter
ax1 = axes[0]
df_mp = df_vote_check.copy()
ax1.scatter(df_mp['valid_votes'], df_mp['sum_cand_votes'], s=25, alpha=0.6,
            color='#2196F3', edgecolors='gray', linewidth=0.3)
max_v = max(df_mp['valid_votes'].max(), df_mp['sum_cand_votes'].max()) * 1.05
ax1.plot([0, max_v], [0, max_v], 'r--', linewidth=1.5, label='เส้น y=x')
ax1.set_xlabel('บัตรดี สส.แบ่งเขต (Valid Votes)')
ax1.set_ylabel('ผลรวมคะแนนผู้สมัคร')
ax1.set_title(f'🗳️ สส.แบ่งเขต: บัตรดี vs คะแนนรวม\n✅ ตรงกันทั้ง {exact_match} เขต (100%)', fontsize=14)
ax1.legend(fontsize=12)
ax1.set_xlim(0, max_v)
ax1.set_ylim(0, max_v)

# Right: Party-list — scatter
ax2 = axes[1]
# Color by diff
if has_diff_pl > 0:
    sc2 = ax2.scatter(df_pl_check['pl_valid_votes'], df_pl_check['sum_party_pl_votes'],
                      c=df_pl_check['diff_pct'], cmap='RdYlGn_r', s=25, alpha=0.6,
                      edgecolors='gray', linewidth=0.3)
    plt.colorbar(sc2, ax=ax2, label='% ผลต่าง')
else:
    ax2.scatter(df_pl_check['pl_valid_votes'], df_pl_check['sum_party_pl_votes'],
                s=25, alpha=0.6, color='#4CAF50', edgecolors='gray', linewidth=0.3)
max_pl = max(df_pl_check['pl_valid_votes'].max(), df_pl_check['sum_party_pl_votes'].max()) * 1.05
ax2.plot([0, max_pl], [0, max_pl], 'r--', linewidth=1.5, label='เส้น y=x')
ax2.set_xlabel('บัตรดี บัญชีรายชื่อ (PL Valid Votes)')
ax2.set_ylabel('ผลรวมคะแนนทุกพรรค')
match_txt = f"✅ ตรงกัน {exact_pl} เขต" if has_diff_pl == 0 else f"🔶 ต่างกัน {has_diff_pl} เขต"
ax2.set_title(f'🗳️ บัญชีรายชื่อ: บัตรดี vs คะแนนรวมพรรค\n{match_txt}', fontsize=14)
ax2.legend(fontsize=12)
ax2.set_xlim(0, max_pl)
ax2.set_ylim(0, max_pl)

plt.suptitle('🔍 ตรวจสอบความสอดคล้อง: บัตรดี vs คะแนนโหวตรวม ทั้ง 2 ระบบ', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# If there are party-list mismatches, show details
if has_diff_pl > 0:
    print("\n📋 เขตที่บัญชีรายชื่อมีผลต่างมากที่สุด:")
    top_pl = df_pl_check.nlargest(15, 'diff_abs')[
        ['province', 'cons_id', 'pl_valid_votes', 'sum_party_pl_votes', 'diff', 'diff_pct']
    ].copy()
    top_pl.columns = ['จังหวัด', 'เขต', 'บัตรดีPL', 'คะแนนรวมพรรค', 'ผลต่าง', '%ผลต่าง']
    print(top_pl.to_string(index=False))
else:
    print("\n✅ ทั้ง สส.แบ่งเขต และ บัญชีรายชื่อ — บัตรดีตรงกับคะแนนรวมทุกเขต!")

🔍 ตรวจสอบ: บัตรดีบัญชีรายชื่อ vs ผลรวมคะแนนทุกพรรค (รายเขต)

📊 จำนวนเขต: 400 เขต
   ✅ ตรงกันพอดี: 400 เขต (100.0%)
   🔶 ต่างกัน:    0 เขต (0.0%)

✅ ทั้ง สส.แบ่งเขต และ บัญชีรายชื่อ — บัตรดีตรงกับคะแนนรวมทุกเขต!


---
## 14. 🔀 วิเคราะห์ Vote-Split: คะแนนแบ่งเขต vs บัญชีรายชื่อ รายพรรค
**คำถาม:** แต่ละพรรคได้คะแนน สส.แบ่งเขต เทียบกับ สส.บัญชีรายชื่อ ต่างกันแค่ไหน?  
พรรคที่คนเลือก "ตัวบุคคล" มากกว่า "พรรค" จะมีคะแนนแบ่งเขต > บัญชีรายชื่อ  
พรรคที่มี "แบรนด์พรรค" แข็งแกร่ง จะมีบัญชีรายชื่อ > แบ่งเขต

In [28]:
# === 14. Vote-Split Analysis: Constituency Votes vs Party-List Votes ===

# party_cons_votes = total constituency votes per party per province (from result_party in each province)
# party_list_vote = party-list votes per party per province

# Extract province-level party data
prov_party_records = []
for prov in province_stats:
    prov_id = prov['prov_id']
    for rp in prov.get('result_party', []):
        prov_party_records.append({
            'prov_id': prov_id,
            'party_id': rp['party_id'],
            'party_cons_votes': rp.get('party_cons_votes', 0),
            'party_list_vote': rp.get('party_list_vote', 0),
            'first_mp_app_count': rp.get('first_mp_app_count', 0),
        })

df_prov_party = pd.DataFrame(prov_party_records)

# Aggregate to national level
df_party_split = df_prov_party.groupby('party_id').agg({
    'party_cons_votes': 'sum',
    'party_list_vote': 'sum',
    'first_mp_app_count': 'sum',
}).reset_index()

# Join party names
df_party_split = df_party_split.merge(
    df_parties[['id', 'name', 'color', 'abbr']].rename(columns={'id': 'party_id'}),
    on='party_id', how='left'
)

# Calculate split ratio
df_party_split['total_votes'] = df_party_split['party_cons_votes'] + df_party_split['party_list_vote']
df_party_split['cons_pct'] = (df_party_split['party_cons_votes'] / df_party_split['total_votes'] * 100).round(2)
df_party_split['pl_pct'] = (df_party_split['party_list_vote'] / df_party_split['total_votes'] * 100).round(2)
df_party_split['split_ratio'] = (df_party_split['party_cons_votes'] / df_party_split['party_list_vote'].replace(0, 1)).round(3)
df_party_split = df_party_split.sort_values('total_votes', ascending=False).reset_index(drop=True)

# Top 15 parties visualization
top15_split = df_party_split.head(15).copy()

fig, axes = plt.subplots(1, 2, figsize=(22, 9))

# --- Left: Stacked bar comparing cons vs PL votes ---
ax1 = axes[0]
y_pos = range(len(top15_split))
bars_c = ax1.barh(y_pos, top15_split['party_cons_votes'], label='คะแนนแบ่งเขต',
                  color=[c if c else '#888' for c in top15_split['color']], edgecolor='white', alpha=0.8)
bars_p = ax1.barh(y_pos, top15_split['party_list_vote'], left=top15_split['party_cons_votes'],
                  label='คะแนนบัญชีรายชื่อ', color=[c if c else '#888' for c in top15_split['color']],
                  edgecolor='white', alpha=0.4, hatch='///')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(top15_split['name'].tolist())
ax1.invert_yaxis()
ax1.set_xlabel('จำนวนคะแนน')
ax1.set_title('🔀 คะแนนแบ่งเขต vs บัญชีรายชื่อ (Top 15 พรรค)', fontsize=14)
ax1.legend(loc='lower right', fontsize=11)

for i, row in top15_split.iterrows():
    total = row['total_votes']
    ax1.text(total + max(top15_split['total_votes']) * 0.01, i,
             f"{total:,.0f}", va='center', fontsize=9)

# --- Right: Split ratio scatter ---
ax2 = axes[1]
colors_split = [c if c else '#888' for c in top15_split['color']]
scatter_sizes = (top15_split['total_votes'] / top15_split['total_votes'].max() * 500 + 50)

for idx, row in top15_split.iterrows():
    ax2.scatter(row['party_list_vote'], row['party_cons_votes'],
                s=scatter_sizes.iloc[idx], c=row['color'] if row['color'] else '#888',
                alpha=0.7, edgecolors='black', linewidth=0.5)
    ax2.annotate(row['name'], (row['party_list_vote'], row['party_cons_votes']),
                 textcoords="offset points", xytext=(8, 4), fontsize=8)

# y=x line (balanced)
max_both = max(top15_split['party_cons_votes'].max(), top15_split['party_list_vote'].max()) * 1.1
ax2.plot([0, max_both], [0, max_both], 'k--', alpha=0.3, label='เส้นสมดุล (cons = PL)')
ax2.set_xlabel('คะแนนบัญชีรายชื่อ')
ax2.set_ylabel('คะแนนแบ่งเขต')
ax2.set_title('🎯 เปรียบเทียบ: พรรคไหนได้ "ตัวบุคคล" > "แบรนด์พรรค"?', fontsize=14)
ax2.legend(fontsize=10)
ax2.text(max_both * 0.05, max_both * 0.9, '⬆️ ตัวบุคคลแข็ง\n(แบ่งเขต > PL)', fontsize=10, color='gray')
ax2.text(max_both * 0.6, max_both * 0.1, '➡️ แบรนด์พรรคแข็ง\n(PL > แบ่งเขต)', fontsize=10, color='gray')

plt.tight_layout()
plt.show()

# Summary table
print("\n📊 Vote-Split Summary (Top 15):")
display_cols = top15_split[['name', 'party_cons_votes', 'party_list_vote', 'cons_pct', 'pl_pct', 'split_ratio', 'first_mp_app_count']].copy()
display_cols.columns = ['พรรค', 'คะแนนแบ่งเขต', 'คะแนน PL', '%แบ่งเขต', '%PL', 'อัตราส่วน(cons/PL)', 'เขตที่ชนะ']
print(display_cols.to_string(index=False))


📊 Vote-Split Summary (Top 15):
           พรรค  คะแนนแบ่งเขต  คะแนน PL  %แบ่งเขต   %PL  อัตราส่วน(cons/PL)  เขตที่ชนะ
        ประชาชน       7864475   9802658     44.51 55.49               0.802         87
      ภูมิใจไทย       9993777   5964814     62.62 37.38               1.675        174
       เพื่อไทย       5789512   5158066     52.88 47.12               1.122         58
   ประชาธิปัตย์       2017120   3662606     35.51 64.49               0.551         10
       กล้าธรรม       3847563    606312     86.39 13.61               6.346         56
       เศรษฐกิจ        366414   1047721     25.91 74.09               0.350          0
รวมไทยสร้างชาติ        293799    706918     29.36 70.64               0.416          0
      ประชาชาติ        288759    401636     41.83 58.17               0.719          4
   เพื่อชาติไทย           459    629628      0.07 99.93               0.001          0
   พลังประชารัฐ        466641    130730     78.12 21.88               3.570          4
      ไทรวม

### 15. Competition Intensity Analysis — สนามเลือกตั้งที่สูสี vs ถล่มทลาย\nวิเคราะห์ส่วนต่างคะแนนระหว่างอันดับ 1 กับอันดับ 2 ในแต่ละเขต เพื่อหาสนามที่แข่งขันสูสีที่สุดและสนามที่ชนะขาด

In [31]:
# === 15. Competition Intensity — Closest Races & Landslide Wins ===

# Get rank 1 and rank 2 candidates per constituency
df_rank1 = df_cand_full[df_cand_full['mp_app_rank'] == 1].copy()
df_rank2 = df_cand_full[df_cand_full['mp_app_rank'] == 2].copy()

# Merge to get margin (use actual column names: mp_app_name, party_name, province)
df_margin = df_rank1[['cons_id', 'mp_app_vote', 'party_name', 'mp_app_name', 'province']].merge(
    df_rank2[['cons_id', 'mp_app_vote', 'party_name', 'mp_app_name']],
    on='cons_id', suffixes=('_1st', '_2nd')
)

# Calculate margin
df_margin['vote_margin'] = df_margin['mp_app_vote_1st'] - df_margin['mp_app_vote_2nd']
df_margin['margin_pct'] = (df_margin['vote_margin'] / df_margin['mp_app_vote_1st'] * 100).round(2)
df_margin['label'] = df_margin['province'].fillna(df_margin['cons_id']) + ' (' + df_margin['cons_id'] + ')'

# Sort
df_margin_sorted = df_margin.sort_values('vote_margin')

# === Display closest races ===
print("=" * 80)
print("🏆 TOP 20 สนามสูสีที่สุด (Closest Races)")
print("=" * 80)
closest = df_margin_sorted.head(20)
for _, row in closest.iterrows():
    print(f"  {row['label']}: {row['party_name_1st']} ({row['mp_app_name_1st']}) "
          f"ชนะ {row['party_name_2nd']} ({row['mp_app_name_2nd']}) "
          f"ด้วยคะแนน {row['vote_margin']:,.0f} ({row['margin_pct']:.1f}%)")

print(f"\n{'=' * 80}")
print("💪 TOP 20 สนามชนะถล่มทลาย (Biggest Landslides)")
print("=" * 80)
landslide = df_margin_sorted.tail(20).iloc[::-1]
for _, row in landslide.iterrows():
    print(f"  {row['label']}: {row['party_name_1st']} ({row['mp_app_name_1st']}) "
          f"ชนะ {row['party_name_2nd']} ({row['mp_app_name_2nd']}) "
          f"ด้วยคะแนน {row['vote_margin']:,.0f} ({row['margin_pct']:.1f}%)")

# === Margin distribution stats ===
print(f"\n{'=' * 80}")
print("📊 Margin Distribution")
print("=" * 80)
print(f"  ส่วนต่างเฉลี่ย: {df_margin['vote_margin'].mean():,.0f} คะแนน ({df_margin['margin_pct'].mean():.1f}%)")
print(f"  ส่วนต่างมัธยฐาน: {df_margin['vote_margin'].median():,.0f} คะแนน ({df_margin['margin_pct'].median():.1f}%)")
print(f"  สนามที่ชนะ < 1,000 คะแนน: {(df_margin['vote_margin'] < 1000).sum()} เขต")
print(f"  สนามที่ชนะ < 5,000 คะแนน: {(df_margin['vote_margin'] < 5000).sum()} เขต")
print(f"  สนามที่ชนะ > 50,000 คะแนน: {(df_margin['vote_margin'] > 50000).sum()} เขต")

# === Visualization ===
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Competition Intensity Analysis', fontsize=14, fontweight='bold')

# 1) Top 15 closest races
ax = axes[0, 0]
close15 = df_margin_sorted.head(15)
colors_close = ['#e74c3c' if m < 1000 else '#f39c12' if m < 3000 else '#3498db' for m in close15['vote_margin']]
ax.barh(range(len(close15)), close15['vote_margin'], color=colors_close)
ax.set_yticks(range(len(close15)))
ax.set_yticklabels([f"{r['label']}" for _, r in close15.iterrows()], fontsize=7)
ax.set_xlabel('Vote Margin')
ax.set_title('Top 15 Closest Races', fontsize=11)
ax.invert_yaxis()
for i, (_, r) in enumerate(close15.iterrows()):
    ax.text(r['vote_margin'] + 50, i, f"{r['vote_margin']:,.0f}", va='center', fontsize=7)

# 2) Top 15 landslide wins
ax = axes[0, 1]
land15 = df_margin_sorted.tail(15).iloc[::-1]
colors_land = ['#27ae60' if m > 80000 else '#2ecc71' if m > 50000 else '#3498db' for m in land15['vote_margin']]
ax.barh(range(len(land15)), land15['vote_margin'], color=colors_land)
ax.set_yticks(range(len(land15)))
ax.set_yticklabels([f"{r['label']}" for _, r in land15.iterrows()], fontsize=7)
ax.set_xlabel('Vote Margin')
ax.set_title('Top 15 Biggest Landslides', fontsize=11)
ax.invert_yaxis()
for i, (_, r) in enumerate(land15.iterrows()):
    ax.text(r['vote_margin'] + 500, i, f"{r['vote_margin']:,.0f}", va='center', fontsize=7)

# 3) Margin distribution histogram
ax = axes[1, 0]
ax.hist(df_margin['vote_margin'], bins=50, color='#3498db', edgecolor='white', alpha=0.8)
ax.axvline(df_margin['vote_margin'].median(), color='red', linestyle='--', label=f"Median: {df_margin['vote_margin'].median():,.0f}")
ax.axvline(df_margin['vote_margin'].mean(), color='orange', linestyle='--', label=f"Mean: {df_margin['vote_margin'].mean():,.0f}")
ax.set_xlabel('Vote Margin (1st - 2nd)')
ax.set_ylabel('Number of Constituencies')
ax.set_title('Distribution of Victory Margins', fontsize=11)
ax.legend(fontsize=8)

# 4) Margin by winning party
ax = axes[1, 1]
top_parties_m = df_margin['party_name_1st'].value_counts().head(8).index
df_party_margin = df_margin[df_margin['party_name_1st'].isin(top_parties_m)]
party_medians = df_party_margin.groupby('party_name_1st')['vote_margin'].median().sort_values(ascending=False)
party_order = party_medians.index.tolist()
bp_data = [df_party_margin[df_party_margin['party_name_1st'] == p]['vote_margin'].values for p in party_order]
bp = ax.boxplot(bp_data, vert=True, patch_artist=True, labels=[p[:10] for p in party_order])
party_colors_box = ['#FF6B35', '#1E88E5', '#E53935', '#43A047', '#FDD835', '#8E24AA', '#00ACC1', '#F4511E']
for patch, color in zip(bp['boxes'], party_colors_box[:len(bp['boxes'])]):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
ax.set_ylabel('Vote Margin')
ax.set_title('Victory Margin by Winning Party', fontsize=11)
ax.tick_params(axis='x', rotation=45, labelsize=8)

plt.tight_layout()
plt.savefig(str(DATA_DIR / 'processed' / 'competition_intensity.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n✅ Saved: competition_intensity.png")

🏆 TOP 20 สนามสูสีที่สุด (Closest Races)
  นราธิวาส (NWT_3): กล้าธรรม (นายสัมพันธ์ มะยูโซ๊ะ) ชนะ ภูมิใจไทย (นายแวรุสลัน มะสาและ) ด้วยคะแนน 113 (0.3%)
  ศรีสะเกษ (SSK_6): ภูมิใจไทย (นายคชศักดิ์ ศิริรัตน์มานะวงศ์) ชนะ เพื่อไทย (นายวีระพล จิตสัมฤทธิ์) ด้วยคะแนน 216 (0.8%)
  แม่ฮ่องสอน (MSN_2): ภูมิใจไทย (นายสมบัติ ยะสินธุ์) ชนะ กล้าธรรม (นายวิทยา หวานซึ้ง) ด้วยคะแนน 287 (1.2%)
  อุดรธานี (UDN_7): เพื่อไทย (นายธีระชัย แสนแก้ว) ชนะ ประชาชน (นายสุริยา วงศ์อารีย์) ด้วยคะแนน 516 (2.1%)
  นครราชสีมา (NMA_3): ประชาชน (นายศุทธสิทธิ์ พจน์ฐศักดิ์) ชนะ เพื่อไทย (นายสมบัติ กาญจนวัฒนา) ด้วยคะแนน 530 (1.5%)
  ขอนแก่น (KKN_3): กล้าธรรม (นายเอกชัย สืบสารคาม) ชนะ ประชาชน (นายชัชวาล อภิรักษ์มั่นคง) ด้วยคะแนน 611 (2.2%)
  ร้อยเอ็ด (RET_4): เพื่อไทย (นายศุภศิษย์ กอเจริญยศ) ชนะ กล้าธรรม (นายนีโอ พลซื่อ) ด้วยคะแนน 679 (3.2%)
  นครราชสีมา (NMA_16): ภูมิใจไทย (นายตติรัฐ รัตนเศรษฐ) ชนะ เพื่อไทย (นายพรเทพ ศิริโรจนกุล) ด้วยคะแนน 849 (2.9%)
  สระบุรี (SRI_1): ภูมิใจไทย (นายขุนทอง แสนวิเศษ) ชนะ ประชาชน (นาายสรพัช ศรีป

### 16. Party-List Candidate Depth & PM Candidates Analysis\nวิเคราะห์ความพร้อมของพรรค: จำนวนผู้สมัคร PM, จำนวนผู้สมัครบัญชีรายชื่อ, และเปรียบเทียบกับผลคะแนน

In [35]:
# === 16. Party-List Candidate Depth & PM Candidates ===

# Parse party_candidate data
party_cand_records = []
for p in raw_party_candidate:
    party_no = p.get('party_no')
    pm_count = len(p.get('pm_candidates', []))
    pl_count = len(p.get('party_list_candidates', []))
    pm_names = [c['name'] for c in p.get('pm_candidates', [])]
    party_cand_records.append({
        'party_no': party_no,
        'pm_candidate_count': pm_count,
        'pl_candidate_count': pl_count,
        'pm_names': ', '.join(pm_names) if pm_names else '-'
    })

df_party_cand = pd.DataFrame(party_cand_records)

# Map party_no to party_name using party overview
party_no_to_name = {}
for p in raw_party:
    party_no_to_name[p.get('party_no')] = p.get('name', f"Party #{p.get('party_no')}")
df_party_cand['party_name'] = df_party_cand['party_no'].map(party_no_to_name).fillna('Unknown')

# Use df_party_full (has party_no + party_vote from join in cell 14)
df_party_cand = df_party_cand.merge(
    df_party_full[['party_no', 'party_vote']].rename(columns={'party_vote': 'total_pl_votes'}),
    on='party_no', how='left'
)

# Also merge constituency wins (df_winners uses 'party_name')
cons_wins = df_winners['party_name'].value_counts().reset_index()
cons_wins.columns = ['party_name', 'constituency_wins']
df_party_cand = df_party_cand.merge(cons_wins, on='party_name', how='left')
df_party_cand['constituency_wins'] = df_party_cand['constituency_wins'].fillna(0).astype(int)
df_party_cand['total_pl_votes'] = df_party_cand['total_pl_votes'].fillna(0).astype(int)

# Sort by total PL votes
df_party_cand = df_party_cand.sort_values('total_pl_votes', ascending=False)

# Display
print("=" * 100)
print("📋 Party Readiness: PM Candidates & Party-List Depth")
print("=" * 100)
print(f"{'พรรค':<20} {'PM':>4} {'PL':>5} {'PL Votes':>12} {'เขตชนะ':>6}  แคนดิเดต PM")
print("-" * 100)
for _, row in df_party_cand.head(25).iterrows():
    pname = str(row['party_name'])[:18]
    pm_str = str(row['pm_names'])[:55]
    print(f"{pname:<20} {row['pm_candidate_count']:>4} {row['pl_candidate_count']:>5} "
          f"{row['total_pl_votes']:>12,} {row['constituency_wins']:>6}  {pm_str}")

# Summary stats
has_pm = df_party_cand[df_party_cand['pm_candidate_count'] > 0]
no_pm = df_party_cand[df_party_cand['pm_candidate_count'] == 0]
print(f"\n📊 Summary:")
print(f"  พรรคที่เสนอชื่อ PM: {len(has_pm)} / {len(df_party_cand)} พรรค")
print(f"  จำนวน PM candidates ทั้งหมด: {df_party_cand['pm_candidate_count'].sum()} คน")
print(f"  จำนวน PL candidates ทั้งหมด: {df_party_cand['pl_candidate_count'].sum()} คน")
max_pl = df_party_cand.loc[df_party_cand['pl_candidate_count'].idxmax()]
print(f"  พรรคที่มี PL มากสุด: {max_pl['party_name']} ({max_pl['pl_candidate_count']} คน)")

# === Visualization ===
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Party-List Candidate Depth & PM Candidates', fontsize=14, fontweight='bold')

# 1) Top 20 parties: PL candidates bar
ax = axes[0, 0]
top20_c = df_party_cand.head(20)
x_pos = range(len(top20_c))
ax.bar(x_pos, top20_c['pl_candidate_count'], color='#3498db', alpha=0.8)
ax.set_xticks(x_pos)
ax.set_xticklabels([str(n)[:8] for n in top20_c['party_name']], rotation=45, ha='right', fontsize=7)
ax.set_ylabel('PL Candidates')
ax.set_title('Party-List Candidates (by vote rank)', fontsize=11)

# 2) PM candidate count distribution
ax = axes[0, 1]
pm_dist = df_party_cand['pm_candidate_count'].value_counts().sort_index()
colors_pm = ['#e74c3c' if v == 0 else '#f39c12' if v == 1 else '#2ecc71' if v == 2 else '#3498db' for v in pm_dist.index]
ax.bar(pm_dist.index, pm_dist.values, color=colors_pm)
ax.set_xlabel('Number of PM Candidates')
ax.set_ylabel('Number of Parties')
ax.set_title('PM Candidate Distribution', fontsize=11)
for idx_val, val in pm_dist.items():
    ax.text(idx_val, val + 0.3, str(val), ha='center', fontweight='bold')

# 3) Scatter: PL candidates vs PL votes
ax = axes[1, 0]
top30_c = df_party_cand[df_party_cand['total_pl_votes'] > 0].head(30)
sc = ax.scatter(top30_c['pl_candidate_count'], top30_c['total_pl_votes'],
                c=top30_c['pm_candidate_count'], cmap='RdYlGn', s=100, edgecolors='black', linewidth=0.5)
plt.colorbar(sc, ax=ax, label='PM Candidates')
for _, row in top30_c.head(10).iterrows():
    ax.annotate(str(row['party_name'])[:10], (row['pl_candidate_count'], row['total_pl_votes']),
                fontsize=6, ha='center', va='bottom')
ax.set_xlabel('Party-List Candidates')
ax.set_ylabel('Party-List Votes')
ax.set_title('PL Candidates vs PL Votes', fontsize=11)

# 4) Has PM vs No PM: average vote comparison
ax = axes[1, 1]
avg_pm = has_pm['total_pl_votes'].mean()
avg_no_pm = no_pm['total_pl_votes'].mean()
med_pm = has_pm['total_pl_votes'].median()
med_no_pm = no_pm['total_pl_votes'].median()
x_labels = ['Has PM\n(Mean)', 'No PM\n(Mean)', 'Has PM\n(Median)', 'No PM\n(Median)']
values = [avg_pm, avg_no_pm, med_pm, med_no_pm]
colors_comp = ['#27ae60', '#e74c3c', '#27ae60', '#e74c3c']
ax.bar(range(4), values, color=colors_comp, alpha=0.8)
ax.set_xticks(range(4))
ax.set_xticklabels(x_labels, fontsize=9)
ax.set_ylabel('Party-List Votes')
ax.set_title('PM Candidacy vs Vote Performance', fontsize=11)
for i, v in enumerate(values):
    ax.text(i, v + max(values)*0.02, f"{v:,.0f}", ha='center', fontsize=8)

plt.tight_layout()
plt.savefig(str(DATA_DIR / 'processed' / 'party_candidate_depth.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n✅ Saved: party_candidate_depth.png")

📋 Party Readiness: PM Candidates & Party-List Depth
พรรค                   PM    PL     PL Votes เขตชนะ  แคนดิเดต PM
----------------------------------------------------------------------------------------------------
Unknown                 1    24    9,802,658      0  กัณวีร์ สืบแสง
Unknown                 3    99    9,802,658      0  ณัฐพงษ์ เรืองปัญญาวุฒิ, ศิริกัญญา ตันสกุล, วีระยุทธ กาญ
Unknown                 2    98    5,964,814      0  อนุทิน ชาญวีรกูล, สีหศักดิ์ พวงเกตุแก้ว
Unknown                 3   100    5,158,066      0  ยศชนัน วงศ์สวัสดิ์, จุลพันธ์ อมรวิวัฒน์, สุริยะ จึงรุ่ง
Unknown                 3    97    3,662,606      0  อภิสิทธิ์ เวชชาชีวะ, กรณ์ จาติกวณิช, การดี เลียวไพโรจน์
Unknown                 2    61    1,047,721      0  รังษี กิตติญาณทรัพย์, คริส โปตระนันทน์
Unknown                 3    98      706,918      0  พีระพันธุ์ สาลีรัฐวิภาค, อรรถวิชช์ สุวรรณภักดี, นราพัฒน
Unknown                 0     6      629,628      0  -
Unknown                 1    96      6

### 17. Province-Level Party Dominance Heatmap\nวิเคราะห์ว่าพรรคไหนครองจังหวัดไหน — เปรียบเทียบ party-list votes ระดับจังหวัดสำหรับพรรคหลัก

In [39]:
# === 17. Province-Level Party Dominance Heatmap ===

# Re-extract province-level party data from raw (to avoid column pollution)
pp_records = []
for prov in raw_stats_cons['result_province']:
    prov_id = prov['prov_id']
    for rp in prov.get('result_party', []):
        pp_records.append({
            'prov_id': prov_id,
            'party_id': rp['party_id'],
            'party_list_vote': rp.get('party_list_vote', 0),
            'party_cons_votes': rp.get('party_cons_votes', 0),
            'first_mp_app_count': rp.get('first_mp_app_count', 0),
        })
df_pp = pd.DataFrame(pp_records)

# Build party_id → name map from raw_party (id is str, party_id from stats is int)
pid_to_name = {}
for p in raw_party:
    pid_to_name[int(p['id']) if isinstance(p['id'], str) else p['id']] = p.get('name', '?')
df_pp['party_name'] = df_pp['party_id'].map(pid_to_name).fillna('Unknown')

# Map prov_id → province Thai name
prov_id_to_name = dict(zip(df_provinces['prov_id'], df_provinces['province']))
df_pp['province'] = df_pp['prov_id'].map(prov_id_to_name).fillna(df_pp['prov_id'])

# Calculate total PL votes per province for %
prov_total = df_pp.groupby('prov_id')['party_list_vote'].sum().reset_index()
prov_total.columns = ['prov_id', 'total_prov_pl_votes']
df_pp = df_pp.merge(prov_total, on='prov_id', how='left')
df_pp['pl_vote_pct'] = (df_pp['party_list_vote'] / df_pp['total_prov_pl_votes'] * 100).round(2)

# Find which party is #1 in each province by PL votes
prov_winner = df_pp.sort_values('party_list_vote', ascending=False).drop_duplicates('prov_id', keep='first')

print("=" * 80)
print("🗺️ Province Party Dominance (Party-List Votes)")
print("=" * 80)
winner_counts = prov_winner['party_name'].value_counts()
for party, count in winner_counts.items():
    provinces = prov_winner[prov_winner['party_name'] == party]['province'].tolist()
    print(f"\n  {party}: ชนะ {count} จังหวัด")
    print(f"    {', '.join([str(p) for p in provinces[:15]])}")
    if len(provinces) > 15:
        print(f"    ... และอีก {len(provinces) - 15} จังหวัด")

# === Heatmap: Top 6 parties × all provinces ===
top6_parties = df_pp.groupby('party_name')['party_list_vote'].sum().nlargest(6).index.tolist()

heatmap_data = df_pp[df_pp['party_name'].isin(top6_parties)].pivot_table(
    index='province', columns='party_name', values='pl_vote_pct', fill_value=0
)
party_cols = [c for c in top6_parties if c in heatmap_data.columns]
heatmap_data = heatmap_data[party_cols]

# Sort provinces by dominant party
heatmap_data['dominant'] = heatmap_data.idxmax(axis=1)
heatmap_data['dominant_pct'] = heatmap_data[party_cols].max(axis=1)
heatmap_data = heatmap_data.sort_values(['dominant', 'dominant_pct'], ascending=[True, False])
heatmap_plot = heatmap_data[party_cols]

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(22, max(16, len(heatmap_plot) * 0.28)),
                          gridspec_kw={'width_ratios': [3, 1]})

# 1) Full heatmap
ax = axes[0]
im = ax.imshow(heatmap_plot.values, cmap='YlOrRd', aspect='auto')
ax.set_yticks(range(len(heatmap_plot)))
ax.set_yticklabels(heatmap_plot.index, fontsize=6)
ax.set_xticks(range(len(party_cols)))
ax.set_xticklabels([p[:10] for p in party_cols], rotation=45, ha='right', fontsize=9)
ax.set_title('Party-List Vote % by Province (Top 6 Parties)', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax, label='Vote %', shrink=0.5)

# 2) Province count won by each party
ax = axes[1]
colors_dom = ['#fd6512', '#0c149c', '#e53935', '#1e88e5', '#f9a825', '#7b1fa2', '#00897b', '#f4511e']
ax.barh(range(len(winner_counts)), winner_counts.values, color=colors_dom[:len(winner_counts)])
ax.set_yticks(range(len(winner_counts)))
ax.set_yticklabels(winner_counts.index, fontsize=9)
ax.set_xlabel('Provinces Won')
ax.set_title('Province Dominance Count', fontsize=12, fontweight='bold')
for i, v in enumerate(winner_counts.values):
    ax.text(v + 0.3, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.savefig(str(DATA_DIR / 'processed' / 'province_party_dominance.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n✅ Saved: province_party_dominance.png")

🗺️ Province Party Dominance (Party-List Votes)

  ประชาชน: ชนะ 39 จังหวัด
    กรุงเทพมหานคร, นครราชสีมา, เชียงใหม่, สมุทรปราการ, ชลบุรี, นนทบุรี, ปทุมธานี, เชียงราย, นครปฐม, อุบลราชธานี, พระนครศรีอยุธยา, ราชบุรี, ลำปาง, ระยอง, ฉะเชิงเทรา
    ... และอีก 24 จังหวัด

  เพื่อไทย: ชนะ 15 จังหวัด
    ขอนแก่น, ร้อยเอ็ด, อุดรธานี, ศรีสะเกษ, สกลนคร, กาฬสินธุ์, มหาสารคาม, นครพนม, ชัยภูมิ, ยโสธร, เลย, หนองคาย, หนองบัวลำภู, บึงกาฬ, มุกดาหาร

  ภูมิใจไทย: ชนะ 10 จังหวัด
    บุรีรัมย์, สุรินทร์, สุพรรณบุรี, เพชรบูรณ์, เพชรบุรี, อุทัยธานี, อ่างทอง, อำนาจเจริญ, ตราด, ระนอง

  ประชาธิปัตย์: ชนะ 9 จังหวัด
    นครศรีธรรมราช, สงขลา, สุราษฎร์ธานี, ตรัง, พัทลุง, ชุมพร, กระบี่, พังงา, สตูล

  ประชาชาติ: ชนะ 3 จังหวัด
    นราธิวาส, ปัตตานี, ยะลา

  กล้าธรรม: ชนะ 1 จังหวัด
    พะเยา

✅ Saved: province_party_dominance.png

✅ Saved: province_party_dominance.png


### 18. Turnout vs Party Performance Correlation\nวิเคราะห์ว่าอัตราการออกมาใช้สิทธิ์มีผลต่อคะแนนพรรคไหนบ้าง — Turnout สูงเป็นประโยชน์กับพรรคไหน?

In [40]:
# === 18. Turnout vs Party Performance Correlation ===
from scipy import stats as sp_stats

# Use df_pp (province-level party data from section 17) + df_prov_stats (province turnout)
# df_prov_stats has: prov_id, turn_out, percent_turn_out, valid_votes, etc.

# Get top 5 parties
top5_national = df_pp.groupby('party_name')['party_list_vote'].sum().nlargest(5).index.tolist()
print(f"Top 5 parties for correlation: {top5_national}")

# Pivot: province × party → PL vote %
df_pp_pct = df_pp[df_pp['party_name'].isin(top5_national)].copy()
prov_party_pivot = df_pp_pct.pivot_table(index='prov_id', columns='party_name', values='pl_vote_pct', fill_value=0)

# Merge with turnout data
turnout_cols = df_prov_stats[['prov_id', 'percent_turn_out']].drop_duplicates()
turnout_cols = turnout_cols[turnout_cols['percent_turn_out'] > 0]  # exclude zero/aggregate rows
corr_df = prov_party_pivot.merge(turnout_cols, left_index=True, right_on='prov_id', how='inner').set_index('prov_id')

print(f"\nProvinces with data: {len(corr_df)}")
print(f"\n{'=' * 80}")
print("📈 Turnout vs Party Vote % — Pearson Correlation")
print("=" * 80)

correlations = {}
for party in top5_national:
    if party in corr_df.columns:
        r, p_val = sp_stats.pearsonr(corr_df['percent_turn_out'], corr_df[party])
        correlations[party] = (r, p_val)
        sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "n.s."
        direction = "⬆️ Turnout สูง = คะแนนเพิ่ม" if r > 0 else "⬇️ Turnout สูง = คะแนนลด"
        print(f"  {party:20s}: r = {r:+.4f}, p = {p_val:.4f} {sig}  {direction}")

# === Visualization ===
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Turnout vs Party Performance (Province-Level)', fontsize=14, fontweight='bold')
party_colors_map = {p['name']: p['color'] for p in raw_party}

for idx, party in enumerate(top5_national):
    ax = axes[idx // 3, idx % 3]
    if party in corr_df.columns:
        x = corr_df['percent_turn_out']
        y = corr_df[party]
        color = party_colors_map.get(party, '#3498db')
        ax.scatter(x, y, c=color, alpha=0.6, s=60, edgecolors='black', linewidth=0.3)
        
        # Regression line
        slope, intercept, r_val, p_val, std_err = sp_stats.linregress(x, y)
        x_line = np.linspace(x.min(), x.max(), 100)
        ax.plot(x_line, slope * x_line + intercept, color='red', linewidth=2, linestyle='--')
        
        r, p = correlations[party]
        sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
        ax.set_title(f'{party}\nr={r:+.3f} {sig}', fontsize=10)
        ax.set_xlabel('Turnout %')
        ax.set_ylabel(f'{party[:8]} Vote %')

# Hide empty subplot if any
if len(top5_national) < 6:
    axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig(str(DATA_DIR / 'processed' / 'turnout_party_correlation.png'), dpi=150, bbox_inches='tight')
plt.show()

# === Summary insight ===
print(f"\n{'=' * 80}")
print("💡 Key Insights:")
print("=" * 80)
for party, (r, p_val) in sorted(correlations.items(), key=lambda x: x[1][0], reverse=True):
    if p_val < 0.05:
        if r > 0.3:
            print(f"  ✅ {party}: Turnout สูงช่วยพรรคนี้อย่างมีนัยสำคัญ (r={r:+.3f})")
        elif r < -0.3:
            print(f"  ❌ {party}: Turnout สูงทำให้คะแนนพรรคนี้ลดลง (r={r:+.3f})")
        else:
            print(f"  ➡️ {party}: มีความสัมพันธ์เล็กน้อย (r={r:+.3f})")
    else:
        print(f"  ⚪ {party}: ไม่มีความสัมพันธ์ที่มีนัยสำคัญ (r={r:+.3f}, p={p_val:.3f})")

print("\n✅ Saved: turnout_party_correlation.png")

Top 5 parties for correlation: ['ประชาชน', 'ภูมิใจไทย', 'เพื่อไทย', 'ประชาธิปัตย์', 'เศรษฐกิจ']

Provinces with data: 77

📈 Turnout vs Party Vote % — Pearson Correlation
  ประชาชน             : r = +0.1921, p = 0.0941 n.s.  ⬆️ Turnout สูง = คะแนนเพิ่ม
  ภูมิใจไทย           : r = -0.1822, p = 0.1127 n.s.  ⬇️ Turnout สูง = คะแนนลด
  เพื่อไทย            : r = -0.5300, p = 0.0000 ***  ⬇️ Turnout สูง = คะแนนลด
  ประชาธิปัตย์        : r = +0.4216, p = 0.0001 ***  ⬆️ Turnout สูง = คะแนนเพิ่ม
  เศรษฐกิจ            : r = -0.3556, p = 0.0015 **  ⬇️ Turnout สูง = คะแนนลด

💡 Key Insights:
  ✅ ประชาธิปัตย์: Turnout สูงช่วยพรรคนี้อย่างมีนัยสำคัญ (r=+0.422)
  ⚪ ประชาชน: ไม่มีความสัมพันธ์ที่มีนัยสำคัญ (r=+0.192, p=0.094)
  ⚪ ภูมิใจไทย: ไม่มีความสัมพันธ์ที่มีนัยสำคัญ (r=-0.182, p=0.113)
  ❌ เศรษฐกิจ: Turnout สูงทำให้คะแนนพรรคนี้ลดลง (r=-0.356)
  ❌ เพื่อไทย: Turnout สูงทำให้คะแนนพรรคนี้ลดลง (r=-0.530)

✅ Saved: turnout_party_correlation.png

💡 Key Insights:
  ✅ ประชาธิปัตย์: Turnout สูงช่วยพรรคนี้อย่างม

In [24]:
# === 12. Election Summary Dashboard ===
from IPython.display import Markdown, display

total_turnout = national.get('turn_out', 0)
pct_turnout = national.get('percent_turn_out', 0)
total_valid = national.get('valid_votes', 0)
total_invalid = national.get('invalid_votes', 0)
total_blank = national.get('blank_votes', 0)
pct_counted = national.get('percent_count', 0)
total_registered = df_provinces['total_registered_vote'].sum()

# Top 3 parties summary
top3_party = df_party_full.head(3)
top3_cons = party_wins.head(3)

summary = f"""
# 📊 สรุปภาพรวมการเลือกตั้ง 69

| ข้อมูล | จำนวน |
|--------|-------|
| 🗳️ ผู้มีสิทธิเลือกตั้ง | **{total_registered:,.0f}** คน |
| ✅ ผู้มาใช้สิทธิ | **{total_turnout:,.0f}** คน (**{pct_turnout:.2f}%**) |
| 📋 บัตรดี | **{total_valid:,.0f}** เสียง |
| ❌ บัตรเสีย | **{total_invalid:,.0f}** เสียง (**{national.get('percent_invalid_votes', 0):.2f}%**) |
| ⚪ ไม่ประสงค์ลงคะแนน | **{total_blank:,.0f}** เสียง (**{national.get('percent_blank_votes', 0):.2f}%**) |
| 📦 นับคะแนนแล้ว | **{pct_counted:.2f}%** ของหน่วยเลือกตั้ง |

### 🏆 Top 3 พรรค (บัญชีรายชื่อ)
| อันดับ | พรรค | คะแนน | % |
|--------|------|-------|---|
| 🥇 | {top3_party.iloc[0]['name']} | {top3_party.iloc[0]['party_vote']:,.0f} | {top3_party.iloc[0]['party_vote_percent']:.2f}% |
| 🥈 | {top3_party.iloc[1]['name']} | {top3_party.iloc[1]['party_vote']:,.0f} | {top3_party.iloc[1]['party_vote_percent']:.2f}% |
| 🥉 | {top3_party.iloc[2]['name']} | {top3_party.iloc[2]['party_vote']:,.0f} | {top3_party.iloc[2]['party_vote_percent']:.2f}% |

### 🏅 Top 3 พรรค (สส.แบ่งเขต)
| อันดับ | พรรค | จำนวนเขตที่ชนะ |
|--------|------|---------------|
| 🥇 | {top3_cons.iloc[0]['party_name']} | {top3_cons.iloc[0]['seats_won']} เขต |
| 🥈 | {top3_cons.iloc[1]['party_name']} | {top3_cons.iloc[1]['seats_won']} เขต |
| 🥉 | {top3_cons.iloc[2]['party_name']} | {top3_cons.iloc[2]['seats_won']} เขต |

---
*ข้อมูลจาก API ของ กกต. (ectreport69.ect.go.th) — อัพเดทล่าสุด: {national.get('last_update', 'N/A')}*
"""

display(Markdown(summary))


# 📊 สรุปภาพรวมการเลือกตั้ง 69

| ข้อมูล | จำนวน |
|--------|-------|
| 🗳️ ผู้มีสิทธิเลือกตั้ง | **52,922,923** คน |
| ✅ ผู้มาใช้สิทธิ | **34,632,581** คน (**65.44%**) |
| 📋 บัตรดี | **31,951,912** เสียง |
| ❌ บัตรเสีย | **1,234,047** เสียง (**3.56%**) |
| ⚪ ไม่ประสงค์ลงคะแนน | **1,446,622** เสียง (**4.18%**) |
| 📦 นับคะแนนแล้ว | **94.33%** ของหน่วยเลือกตั้ง |

### 🏆 Top 3 พรรค (บัญชีรายชื่อ)
| อันดับ | พรรค | คะแนน | % |
|--------|------|-------|---|
| 🥇 | ประชาชน | 9,802,658 | 28.30% |
| 🥈 | ภูมิใจไทย | 5,964,814 | 17.22% |
| 🥉 | เพื่อไทย | 5,158,066 | 14.89% |

### 🏅 Top 3 พรรค (สส.แบ่งเขต)
| อันดับ | พรรค | จำนวนเขตที่ชนะ |
|--------|------|---------------|
| 🥇 | ภูมิใจไทย | 174 เขต |
| 🥈 | ประชาชน | 87 เขต |
| 🥉 | เพื่อไทย | 58 เขต |

---
*ข้อมูลจาก API ของ กกต. (ectreport69.ect.go.th) — อัพเดทล่าสุด: 2026-02-10T10:45:50.294Z*


---
## 13. 💾 Export ข้อมูลที่ประมวลผลแล้ว

In [25]:
# === 13. Export Processed Data ===
export_dir = Path("../data/ect_api/processed")
export_dir.mkdir(parents=True, exist_ok=True)

# Export key DataFrames
exports = {
    'candidate_results_full.csv': df_cand_full,
    'party_list_results.csv': df_party_full,
    'province_turnout.csv': df_turnout,
    'constituency_stats.csv': df_cons_stats,
    'party_constituency_wins.csv': party_wins,
    'combined_seats_estimation.csv': df_combined,
}

for filename, df in exports.items():
    filepath = export_dir / filename
    df.to_csv(filepath, index=False, encoding='utf-8-sig')
    print(f"💾 {filename}: {len(df)} rows → {filepath}")

# Metadata
pct_counted = national.get('percent_count', 0)
meta = {
    'exported_at': datetime.now().isoformat(),
    'source': 'กกต. ECT Report API (ectreport69.ect.go.th)',
    'percent_counted': pct_counted,
    'files': {name: len(df) for name, df in exports.items()},
}
with open(export_dir / 'export_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print(f"\n✅ Export สำเร็จ! ทั้งหมดอยู่ที่: {export_dir.resolve()}")
print("📌 สามารถนำไฟล์ CSV ไปเปิดใน Power BI, Tableau หรือ Google Data Studio ได้เลย")

💾 candidate_results_full.csv: 3491 rows → ../data/ect_api/processed/candidate_results_full.csv
💾 party_list_results.csv: 57 rows → ../data/ect_api/processed/party_list_results.csv
💾 province_turnout.csv: 77 rows → ../data/ect_api/processed/province_turnout.csv
💾 constituency_stats.csv: 401 rows → ../data/ect_api/processed/constituency_stats.csv
💾 party_constituency_wins.csv: 10 rows → ../data/ect_api/processed/party_constituency_wins.csv
💾 combined_seats_estimation.csv: 57 rows → ../data/ect_api/processed/combined_seats_estimation.csv

✅ Export สำเร็จ! ทั้งหมดอยู่ที่: /Users/teerapatcheung/Desktop/0-Project/vending-machine-ttfts/election_69_analyzer/data/ect_api/processed
📌 สามารถนำไฟล์ CSV ไปเปิดใน Power BI, Tableau หรือ Google Data Studio ได้เลย
